In [ ]:
import torch # Importa biblioteca do pytorch
from IPython.display import Image # Importa biblioteca para printar imagens

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # importa arquivos do google drive para o disco da máquina virtual

In [ ]:
!mkdir imagens
!unzip [CAMINHO_IMAGENS_ZIPADAS] -d /content/imagens/ # descompacta imagens a serem utilizadas no treinamento/teste/validação
!git clone https://github.com/ultralytics/yolov5 # importa yolov5

In [ ]:
%cd yolov5
!pip install -r requirements.txt # instala bibliotecas necessárias para utilizar o yolov5
!pip install IProgress

In [ ]:
from tqdm import tqdm
from ipywidgets import IntProgress

In [ ]:
torch.cuda.get_arch_list()

In [ ]:
print('Versão Pytorch: %s com %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Treinamento do Modelo YoloV5


 > Copiar arquivos para treinamento.
 
 > Copiar arquivo dataset.yaml para o Diretório: yolov5/data.

In [ ]:
!python train.py --img 256 --rect --batch 64 --epochs 50 --data [CAMINHO]/dataset.yaml --weights yolov5m.pt --cache
!cp -r /content/yolov5/runs/train/exp/ -d [CAMINHO]
from google.colab import runtime
runtime.unassign()

> Copia resultados do treinamento para pasta no google drive

In [ ]:
!cp -r /content/yolov5/runs/train/exp/ -d [CAMINHO]

# Testando o Modelo Treinado:

In [ ]:
Image(filename='runs/train/exp/val_batch0_labels.jpg', width=256)

In [ ]:
!python detect.py --weights [CAMINHO]/best_yolov5m_seg.pt --img 256 --conf 0.25 --source /content/imagens/testing/images

In [ ]:
!cp -r /content/yolov5/runs/detect/exp/ -d [CAMINHO]

> Testando modelo treinado de modo a obter as posições das bounding boxes e o grau de confiança de cada uma.

In [ ]:
dir = '/content/imagens/'
filename = '[CAMINHO]/filename_plates_infer.txt'
temp = open(filename, 'r').read().splitlines()

im = [dir + x for x in temp]

In [ ]:
# inferência para obter coordenadas

# ANOTA PREDIÇÕES
%cd /content/
with open('resultados.txt', 'w') as f:
    f.write('Resultados')
f.close()

model = torch.hub.load('ultralytics/yolov5', 'custom', '[CAMINHO]/best.pt')  # custom trained model

for i in range(0, len(im)):
 results = model(im[i], size = 256)

 with open('resultados.txt', 'a') as f:
  f.write('\n')
  f.writelines('Imagem ')
  f.writelines(str(i+1))
 f.close()

 stringcat = ''
 for j in range(0, len(results.xyxy[0])):
  stringcat = ''
  for k in range(0, 4):
   stringcat = stringcat + ' ' + str(round(float(results.xyxy[0][j][k])))
  stringcat = stringcat + ' ' +  str(float(results.xyxy[0][j][4]))
  with open('resultados.txt', 'a') as f:
     f.write('\n')
     f.writelines(stringcat)
  f.close()

In [ ]:
print(results.xyxy[0][0][0])
x = results.xyxy[0][0][0]
print(int(x))

# Convertendo do formato pytorch para o formato TFLITE para poder utilizar em smartphones

In [ ]:
# Converte para formato fp16 TFlite

!python export.py --weights [CAMINHO]/best.pt --include tflite --data dataset.yaml --img 256

In [ ]:
# Testando Modelo FP16

!python detect.py --weights [CAMINHO]best-fp16.tflite --img 256 --conf 0.25 --data dataset.yaml --source /content/imagens/testing/images

In [ ]:
!cp -r /content/yolov5/runs/detect/exp2/ -d /content/drive/MyDrive/Dados/testes/

# Converte para formato int8

In [ ]:
!python export.py --weights [CAMINHO]/best_yolov5m_seg.pt --include tflite --int8 --img 256 --data dataset.yaml

# AVALIANDO PERFORMANCE

In [ ]:
# INICIALIZA RECURSOS NECESSÁRIOS PARA TESTAR O MODELO

import torch # Importa biblioteca do pytorch
from google.colab import drive
drive.mount('/content/drive') # importa arquivos do google drive para o disco da máquina virtual
!mkdir imagens
!unzip [CAMINHO ARQUIVO ZIP] -d /content/imagens/ # descompacta imagens a serem utilizadas no treinamento/teste/validação
!git clone https://github.com/ultralytics/yolov5 # importa yolov5
%cd yolov5
!pip install -r requirements.txt # instala bibliotecas necessárias para utilizar o yolov5
!pip install IProgress
from tqdm import tqdm
import IProgress
from ipywidgets import IntProgress

In [ ]:
!python val.py --weights [CAMINHO]/best_yolov5m_seg.pt --data segmentation.yaml --img 256 --task test

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', '[CAMINHO]/best_yolov5m_seg.pt')

In [ ]:
dir = '[CAMINHO IMAGENS TESTE]'
filename = '[CAMINHO ARQUIVO COM O NOME DAS IMAGENS]/FilenamePlacaAvalGeral.txt'
temp = open(filename, 'r').read().splitlines()

im = [dir + x for x in temp]

for i in range(0, len(im)):

 results = model(im[i], size = 256)
 a=1
 if results.xyxy[0].numel() == 0:
  bboxes = [i+1, a, im[i][32:45]+'.png', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
  with open('deteccoes_model_aval_geral.txt', 'a') as f:
    f.write('\n')
    f.writelines(str(bboxes))
  f.close()
  continue
 for j in range(0, len(results.xyxy[0])):
    tensor = results.xyxy[0][j]
    xi = tensor[0]
    xf = tensor[2]
    yi = tensor[1]
    yf = tensor[3]
    cf = tensor[4]
    cf = cf.item()
    cf = str(cf)
    cf = cf[0:5]
    cf = float(cf)
    xi = int(round(xi.item(), 0))
    xf = int(round(xf.item(), 0))
    yi = int(round(yi.item(), 0))
    yf = int(round(yf.item(), 0))
    bboxes = [i+1, a, im[i][64:81], yi, yf, xi, xf, cf]
    print(bboxes)
    a = a + 1

    with open('deteccoes_model_aval_geral.txt', 'a') as f:
        f.write('\n')
        f.writelines(str(bboxes))
    f.close()

In [ ]:
print(im[1][64:81])